In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [4]:
num_samples = 1000

In [45]:
import random
import math

# yes, there are more num_samples than unique items in the dataset
# made to keep the problem as simple as possible, but please expirement with other configurations!
def generate_item():
  x = random.randint(1,500)
  y = math.log(x) + 3
  return x, y

In [46]:
inputs, targets = zip(*[generate_item() for _ in range(num_samples)])
inputs = torch.tensor(inputs, dtype = torch.float)
targets = torch.tensor(targets, dtype = torch.float)

In [47]:
print(inputs[0])
print(targets[0])

tensor(1596.)
tensor(10.3753)


In [48]:
inputs = inputs.unsqueeze(1)
targets = targets.unsqueeze(1)
inputs[0]

tensor([1596.])

In [49]:
targets[:5]

tensor([[10.3753],
        [ 9.3869],
        [ 9.4922],
        [ 9.5191],
        [10.4031]])

In [50]:
!pip install datasets

In [51]:
from datasets import Dataset
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split


dataset = TensorDataset(inputs, targets)
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [59]:
model = nn.Sequential(
      nn.Linear(1,64),
      nn.ReLU(),
      nn.Linear(64,128),
      nn.ReLU(),
      nn.Linear(128,256),
      nn.ReLU(),
      nn.Linear(256,1)
)

In [60]:
batch =  next(iter(train_loader))
inputs, targets = batch
print(inputs.shape)
print(targets.shape)


torch.Size([32, 1])
torch.Size([32, 1])


In [61]:
print(model)

Sequential(
  (0): Linear(in_features=1, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=256, bias=True)
  (5): ReLU()
  (6): Linear(in_features=256, out_features=1, bias=True)
)


In [62]:
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr = 3e-5)

#Training loop
num_epochs = 500
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
  model.train()
  running_loss = 0.0
  for inputs, labels in tqdm(train_loader):
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
  print(f'Epoch {epoch +1}, Loss: {running_loss / len(train_loader)}')


  '''
   * this -was- a note to self that I forgot to do lol. But now it's an exercise! see if you can...
     update this to accumalate absolute error and find average - (update for regression, not classification)
     (you should be removing the torch.max, and replacing the 'val_accuracy' w/ mean squared error )
  '''
  #Validation
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for inputs, labels in val_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      _, predicted = torch.max(outputs.data,1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  val_accuracy = correct/total
  print(f'Epoch [{epoch +1} / {num_epochs}] - Validation Accuracy : {val_accuracy * 100:.2f}%')

print('Finished Training')

100%|██████████| 25/25 [00:00<00:00, 494.31it/s]


Epoch 1, Loss: 22.50965118408203
Epoch [1 / 500] - Validation Accuracy : 0.00%


100%|██████████| 25/25 [00:00<00:00, 541.65it/s]


Epoch 2, Loss: 17.76033695220947
Epoch [2 / 500] - Validation Accuracy : 0.00%


100%|██████████| 25/25 [00:00<00:00, 530.15it/s]

...

Epoch 499, Loss: 0.13636504650115966


Epoch [499 / 500] - Validation Accuracy : 0.00%


100%|██████████| 25/25 [00:00<00:00, 455.91it/s]

Epoch 500, Loss: 0.166123625934124


Epoch [500 / 500] - Validation Accuracy : 0.00%
Finished Training


In [63]:
model(torch.tensor([56], dtype = torch.float, device = device))

tensor([7.7193], device='cuda:0', grad_fn=<ViewBackward0>)

In [ ]:
import matplotlib.pyplot as plt
x = [ n for n in range(1, 501)]
outputs = [ model(torch.tensor([n], dtype = torch.float, device = device)).cpu().detach().numpy() for n in x]
targets = [ math.log(n) + math.sqrt(n * 3) for n in x]
plt.plot(x,outputs)
plt.plot(x,targets)
plt.show()